# MPR121 Data Recording Notebook

Author: Christopher Parker (parkecp@mail.uc.edu)

I've tried to make this as user-friendly as possible, but feel free to reach out with any questions.

# Imports, Board Configuration and Function Definitions

In [1]:
# Basic libraries
import os
os.environ['BLINKA_MPR121'] = '1'
os.environ['BLINKA_FT232H'] = '1'
import time
import datetime
from collections import deque

# For reading/writing data to H5 and CSV files and
# displaying tabular info
import h5py
import pandas as pd
import numpy as np
from panel.widgets import Tabulator
import panel as pn
pn.extension('tabulator')

# Libraries for FTDI and I2C
from pyftdi.i2c import I2cController
from pyftdi.usbtools import UsbTools

# Multiprocessing and Asynchronous Execution
from concurrent.futures import ThreadPoolExecutor
import asyncio

# Widgets
import ipywidgets as widgets
from IPython.display import display

# Graphing (for test buttons)
import matplotlib.pyplot as plt

## Device Configuration and Constant Declarations

Before we start recording, we need to configure the MPR121s.

Normally, this would be done automatically by the Adafruit Blinka package, but we cannot use that (because it only allows 1 FT232H at a time).
So instead, we have to use the pyftdi package (which is what Adafruit Blinka uses, also) and manually determine which registers need to be written/read.

Before we start recording, we need to configure the MPR121s.

Normally, this would be done automatically by the Adafruit Blinka package, but we cannot use that (because it only allows 1 FT232H at a time). So instead, we have to use the pyftdi package (which is what Adafruit Blinka uses, also) and manually determine which registers need to be written/read.

In [2]:
# Register Addresses
SOFT_RESET = 0x80
CONFIG = 0x5E
DATA = 0x04

# How many sensor samples we want to store before writing
HISTORY_SIZE = 100

# 12 channels per MPR121, just defining a constant
# for clarity. We are actually only using 6 at the moment,
# but we may change that later on
NUM_CHANNELS = 6

# Filename for saving data (placeholder, will be set below)
filename = ""

In [3]:
# Find all devices with the given vendor and product IDs
# These correspond to the FT232H
devices = UsbTools.find_all([(0x0403, 0x6014)])

NUM_BOARDS = len(devices)
NUM_SENSORS_TOTAL = NUM_BOARDS*NUM_CHANNELS

In [4]:
devices

[(UsbDeviceDescriptor(vid=1027, pid=24596, bus=0, address=255, sn='FT232H1', index=None, description='\uffff\uffff\uffff\uffff\uffff\uffff'),
  1)]

In [5]:
# We will make lists of the I2C controller objects and port objects
# for each FT232H device
i2c_controllers = {}
serial_numbers = [dev.sn for (dev,_) in devices]
for sn in serial_numbers:
    i2c_controllers[sn] = {}
    url = f"ftdi://ftdi:232h:{sn}/1"
    controller = I2cController()
    controller.configure(url)
    i2c_controllers[sn]["controller"] = controller
    port = controller.get_port(0x5A)
    # I really need to figure out a better way to determine the correct address,
    # because this looks silly
    try: # Try to read a byte to see if that's the right address
        port.read_from(0x04, 1)
    except: # If not, try again
        port = controller.get_port(0x5B)
    try:
        port.read_from(0x04, 1)
    except:
        port = controller.get_port(0x5C)
    try:
        port.read_from(0x04, 1)
    except:
        port = controller.get_port(0x5D)
    try:
        port.read_from(0x04,1)
    except:
        print('None of the MPR121 addresses are readable, is the FT232H in I2C mode?')
    i2c_controllers[sn]["port"] = port

In [6]:
# Loop over the FT232Hs, reset and configure each
for sn, val in i2c_controllers.items():
    port = val["port"]
    # Write to Soft Reset Register (0x63 sends reset command)
    port.write_to(SOFT_RESET, b'\x63')
    # 0x5E is the configuration register, setting to 0x8F starts the MPR121
    # with the config used in the Adafruit library (if needed, I can figure
    # out alternative configurations)
    port.write_to(CONFIG, b'\x8F')
    
    # If we don't sleep here, it doesn't have time to start reading properly
    time.sleep(0.1)
    
    # Test that we have started the MPR121 (if it's not properly started,
    # reading from DATA will give an empty bytearray, so we check if that's
    # what we got back)
    cap = port.read_from(DATA, 24)
    if cap != bytearray(24):
        print(f'Board {sn} Configured')

Board FT232H1 Configured


In [7]:
serial_number_sensor_map = {
    "FT232H0": [1, 2, 3, 7, 8, 9],
    "FT232H1": [4, 5, 6, 10, 11, 12],
    "FT232H2": [13, 14, 15, 19, 20, 21],
    "FT232H3": [16, 17, 18, 22, 23, 24],
}

In [8]:
def map_from_sensor_id_to_sn(sensor_id):
    # Figure out which board we are on by mapping from sensor number to board serial number
    sn_idx = [sensor_id in sensors for sensors in serial_number_sensor_map.values()]
    sn = str(np.array(list(serial_number_sensor_map.keys()))[sn_idx].item())
    return sn

In [9]:
# Set up a list of sensor numbers by serial number. I am going to ensure that
# the FT232H devices mounted on the rack are in the order FT232H0, FT232H1, FT232H2, FT232H3,
# with H0 and H1 being aligned above H2 and H3. That means that if we call
# the top shelf of the rack as sensors 1-6, we have the following sensors assigned to each
# board (because each board has 3 cages above and 3 below):
for sn,sensors in serial_number_sensor_map.items():
    try:
        i2c_controllers[sn]["sensors"] = sensors
        print(f"Device with serial {sn} controls sensors: {sensors}")
    except KeyError as e:
        print(f"No device with serial {str(e)} found")
        continue

No device with serial 'FT232H0' found
Device with serial FT232H1 controls sensors: [4, 5, 6, 10, 11, 12]
No device with serial 'FT232H2' found
No device with serial 'FT232H3' found


## Recording Function and Loop

In [10]:
def record(i2c_port, serial_number):
    """
    Reads 24 bytes (2 bytes per channel for 12 channels) from an MPR121 sensor 
    via the given pyftdi I2C port, and returns timestamp and capacitance data.
    """
    local_time_data = deque(maxlen=NUM_CHANNELS)
    local_cap_data = deque(maxlen=NUM_CHANNELS)

    # Read 24 bytes (2 bytes for each of the 12 channels).
    raw_buffer = i2c_port.read_from(DATA, 24)
    if NUM_CHANNELS == 6: # only recording from one channel per cage
        for chan in range(12):
            # We are only recording every other sensor, starting with 1
            if sensor%2 == 0:
                continue
            value = raw_buffer[2 * chan] | (raw_buffer[2 * chan + 1] << 8)
            local_cap_data.append(value)
            local_time_data.append(time.time())
    for chan in range(12):
        # Combine two bytes (little-endian)
        value = raw_buffer[2 * chan] | (raw_buffer[2 * chan + 1] << 8)
        local_cap_data.append(value)
        local_time_data.append(time.time())

    return local_time_data, local_cap_data, serial_number

In [11]:
async def record_sensors():
    """
    Asynchronous function for recording from all sensors and periodically
    writing to file. This is done asynchronously so that we are not blocking
    user inputs while recording (so the user can start/stop and add other data
    to the H5 file without stopping everything).
    """
    board_time_data = {
        sn: {sensor: deque(maxlen=HISTORY_SIZE) for sensor in i2c_controllers[sn]["sensors"]}
        for sn in i2c_controllers.keys()
    }
    board_cap_data = {
        sn: {sensor: deque(maxlen=HISTORY_SIZE) for sensor in i2c_controllers[sn]["sensors"]}
        for sn in i2c_controllers.keys()
    }
    # Track how many reads we've done so that we can write at the appropriate
    # intervals
    loop_ctr = 0
    
    with ThreadPoolExecutor(max_workers=NUM_BOARDS) as executor:
        # Create a group for each board and initialize datasets.
        with h5py.File(filename, "w") as h5f:
            board_groups = {}
            for sn in i2c_controllers.keys():
                grp = h5f.create_group(f"board_{sn}")
                for sensor in i2c_controllers[sn]["sensors"]:
                    grp.create_group(f"sensor_{sensor}")
        while recording_all:
            # Because the while loop is computing stuff pretty much constantly,
            # it blocks the main execution thread for the notebook unless we
            # await for a moment here. This gives control back to the notebook,
            # and checks if the user has pressed any buttons since the last loop
            await asyncio.sleep(0)
            # Launch parallel sensor reads on all I2C ports.
            futures = [executor.submit(record, val["port"], key) for key,val in i2c_controllers.items()]
            results = [future.result() for future in futures]  # Each result is (local_time_data, local_cap_data, serial_number)
            # Append the data from each board to its corresponding deques.
            for local_time, local_cap, serial_number in results:
                for idx,sensor in enumerate(i2c_controllers[serial_number]["sensors"]):
                    board_time_data[serial_number][sensor].append(local_time[idx])
                    board_cap_data[serial_number][sensor].append(local_cap[idx])
            if loop_ctr == HISTORY_SIZE:
                with h5py.File(filename, "r+") as h5f:
                    for sn in i2c_controllers.keys():
                        for sensor in i2c_controllers[sn]["sensors"]:
                            group = h5f[f"board_{sn}"][f"sensor_{sensor}"]
                            group.create_dataset("time_data", data=board_time_data[sn][sensor], chunks=(HISTORY_SIZE,), maxshape=(None,))
                            group.create_dataset("cap_data", data=board_cap_data[sn][sensor], chunks=(HISTORY_SIZE,), maxshape=(None,))
            elif loop_ctr != 0 and loop_ctr%HISTORY_SIZE == 0:
                tmp_ctr = loop_ctr - HISTORY_SIZE
                with h5py.File(filename, "r+") as h5f:
                    for sn in i2c_controllers.keys():
                        for sensor in i2c_controllers[sn]["sensors"]: 
                            group = h5f[f"board_{sn}"][f"sensor_{sensor}"]
                            group["time_data"].resize((tmp_ctr + HISTORY_SIZE,))
                            group["cap_data"].resize((tmp_ctr + HISTORY_SIZE,))
                            group["time_data"][tmp_ctr:tmp_ctr + HISTORY_SIZE] = board_time_data[sn][sensor]
                            group["cap_data"][tmp_ctr:tmp_ctr + HISTORY_SIZE] = board_cap_data[sn][sensor]
            loop_ctr += 1

## Widget Callback Functions

In [12]:
# --- Callback for the Main Start/Stop Button ---
def start_stop_all(button):
    global recording_all, recording_task, filename
    if not recording_all:
        recording_all = True
        # Define the filename here so that another recording session can
        # start without rerunning everything (although it would still be a
        # good idea, I'm doing this in case of user error)
        filename = f"raw_data_{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.h5"
        button.description = "Stop Recording"
        recording_task = asyncio.create_task(record_sensors())
        with output_area:
            print("Started recording sensors asynchronously.")
    else:
        recording_all = False
        button.description = "Start Recording"
        with output_area:
            print("Stopping recording sensors.")
        if recording_task is not None:
            recording_task.cancel()
        # Make sure all sensor buttons are switched to the stopped state
        # (this will also trigger saving the stop_time for each sensor
        # that was not switched off manually)
        for row in sensor_rows.children:
            # row will be an HBox containing GridBoxes
            for grid in row.children:
                for w in grid.children:
                    # For each child widget in the grid, check if it's the toggle button
                    # if it is, set it to stop
                    if isinstance(w, widgets.widget_bool.ToggleButton):
                        w.value = False
                        sensor_id = w.sensor_id
                        sn = map_from_sensor_id_to_sn(sensor_id)
                    # if the widget is a bounded float txt box, check if it's
                    # start or stop volume and set those accordingly
                    elif isinstance(w, widgets.widget_float.BoundedFloatText):
                        if w.layout.grid_area[:4] != "stop":
                            start_vol = w.value
                        else:
                            stop_vol = w.value
                    else:
                        continue
                with h5py.File(filename, "r+") as h5f:
                    # Try/except here in case the user never started/stopped some sensor and a group
                    # was never created for it.
                    try:
                        _ = h5f[f"board_{sn}"]
                    except KeyError as e:
                        print(f"KeyError: {e}")
                        continue
                    try:
                        h5f[f"board_{sn}"][f"sensor_{sensor_id}"].create_dataset("start_vol", data = start_vol)
                        h5f[f"board_{sn}"][f"sensor_{sensor_id}"].create_dataset("stop_vol", data = stop_vol)
                    except KeyError:
                        h5f[f"board_{sn}"].create_group(f"sensor_{sensor_id}")
                        h5f[f"board_{sn}"][f"sensor_{sensor_id}"].create_dataset("start_vol", data = start_vol)
                        h5f[f"board_{sn}"][f"sensor_{sensor_id}"].create_dataset("stop_vol", data = stop_vol)

In [13]:
# --- Timer functions ---
timers_start_time = {}  # Dictionary to store start times for each sensor
# --- Callback for Individual Sensor Start/Stop ---
def sensor_recording(sensor_id, starting, timer_label):
    # Check if the recording has been started and a file was created
    if not os.path.exists(filename):
        with output_area:
            print("Please start the global recording first.")
            return
    global start_except_ctr, stop_except_ctr
    with output_area:
        sn = map_from_sensor_id_to_sn(sensor_id)
        if starting:
            print(f"Sensor {sensor_id}: Recording start triggered.")
            # Set the board ID number based on the sensor ID,
            # sensors 0-11 are on board 0, etc.
            
            timers_start_time[sensor_id] = time.time()
            asyncio.create_task(update_timer(sensor_id, timer_label))  # Start the async task for timer updates
            with h5py.File(filename, "r+") as h5f:
                try:
                    h5f[f"board_{sn}"][f"sensor_{sensor_id}"].create_dataset("start_time", data=time.time())
                except ValueError as e:
                    start_except_ctr += 1
                    print("Attempted to re-record the start time after it has been set")
                    h5f[f"board_{sn}"][f"sensor_{sensor_id}"].create_dataset(f"start_time{start_except_ctr}", data=time.time())
        else:
            print(f"Sensor {sensor_id}: Recording stop triggered.")
            elapsed_time = time.time() - timers_start_time[sensor_id]
            formatted_time = time.strftime('%H:%M:%S', time.gmtime(elapsed_time))
            timer_label.value = f"Time: {formatted_time}"
            del timers_start_time[sensor_id]  # Stop the async task for timer updates
            with h5py.File(filename, "r+") as h5f:
                try:
                    h5f[f"board_{sn}"][f"sensor_{sensor_id}"].create_dataset("stop_time", data=time.time())
                except ValueError as e:
                    stop_except_ctr += 1
                    print("Attempted to re-record the stop time after it has been set")
                    h5f[f"board_{sn}"][f"sensor_{sensor_id}"].create_dataset(f"stop_time{stop_except_ctr}", data=time.time())


async def update_timer(sensor_id, timer_label):
    """Update the timer for the given sensor asynchronously every 5 seconds."""
    while sensor_id in timers_start_time:
        elapsed_time = time.time() - timers_start_time[sensor_id]
        formatted_time = time.strftime('%H:%M:%S', time.gmtime(elapsed_time))
        timer_label.value = f"Time: {formatted_time}"
        await asyncio.sleep(2)  # Wait for 5 seconds before updating again


In [14]:
# --- Callback for Individual Sensor Test Buttons ---
def sensor_test(button):
    with graph_area:
        graph_area.clear_output()
        print(f"Sensor {button.sensor_id}: Test triggered - plotting raw data for confirmation.")
        sn = map_from_sensor_id_to_sn(button.sensor_id)
        try:
            with h5py.File(filename, "r") as h5f:
                last_5sec = h5f[f"board_{sn}"][f"sensor_{button.sensor_id}"]["cap_data"][-250:]
                fig, ax = plt.subplots()
                # Plotting the data; using the index as the x-axis values
                ax.plot(range(len(last_5sec)), last_5sec)
                ax.set_xlabel("Index")
                ax.set_ylabel("Sensor Data")
                ax.set_title(f"Sensor {button.sensor_id}: Last ~5 sec of raw data")
                plt.show()
        except KeyError:
            print("Please wait until at least 250 reads have been completed before testing!")

In [15]:
# --- Callback for the Clear Output/Graph Buttons ---
def clear_output_callback(button):
    output_area.clear_output()
def clear_graph_callback(button):
    graph_area.clear_output()

# Start Here

### Cage Layout

This is laid out as though you are facing the rack.

NaN => No animal on this sensor

In [16]:
csv_file = "layouts/test_layout.csv"  # Path to the CSV file containing the layout
# TODO: Make this a user input

# Load the CSV file into a pandas DataFrame
layout_df = pd.read_csv(csv_file, header=None)

Tabulator(layout_df, width=1200, widths=1200//6, show_index=False)

Tabulator(show_index=False, value=     0    1   ..., width=1200, widths=200)

In [17]:
# Alternative option for display
csv_file = "layouts/test_layout.csv"
layout_df = pd.read_csv(csv_file, header=None)

# Just loop over all entries in the csv and print them in order with sensor numbers (1-indexed)
for row_idx,row in enumerate(layout_df.iterrows()):
    for col_idx,val in enumerate(row[1]):
        print(f"Sensor {row_idx * 6 + col_idx + 1}: {val}")

Sensor 1: A11
Sensor 2: A12
Sensor 3: A13
Sensor 4: A14
Sensor 5: A15
Sensor 6: nan
Sensor 7: A21
Sensor 8: A22
Sensor 9: A23
Sensor 10: A24
Sensor 11: A25
Sensor 12: A26
Sensor 13: A31
Sensor 14: A32
Sensor 15: A33
Sensor 16: A34
Sensor 17: A35
Sensor 18: A36
Sensor 19: A41
Sensor 20: A42
Sensor 21: A43
Sensor 22: A44
Sensor 23: A45
Sensor 24: A46


### Recording UI

In [18]:
# Define widgets

# Global flags and variables for async recording.
recording_all = False
recording_task = None

# Counters to check if we are setting multiple start/end times
# This is probably not necessary, but just in case someone makes a mistake and clicks
# the start or stop on the wrong sensor, we can allow saving multiple start and stop times
# instead of forcing the user to start over. It'll be a bit confusing later on, is the
# issue I see, if there are multiple start times and it's not carefully recorded which
# was correct.
start_except_ctr = 0
stop_except_ctr = 0

# --- Create the Output Widgets for Text and Graphing ---
output_area = widgets.Output(layout=widgets.Layout(overflow_y='scroll', height='500px', width='500px'))
graph_area = widgets.Output(layout=widgets.Layout(overflow_y='scroll', height='500px'))
outputs_hbox = widgets.HBox([
    output_area, graph_area,  
])

# --- Create the Main Start/Stop Button ---
all_devices_button = widgets.Button(
    description="Start Recording",
    button_style='primary',
    layout=widgets.Layout(width='400px', height='60px')
)
all_devices_button.on_click(start_stop_all)

# --- Create buttons grouped by FT232H board serial number ---
button_groups = []
timers = []  # To keep track of timers for each sensor

# 1-indexed loop
for sensor_id in range(1,25):
    # Create the sensor toggle button (starts as "Start").
    toggle_btn = widgets.ToggleButton(
        value=False,
        description=f"Sensor {sensor_id}: Start",
        layout=widgets.Layout(grid_area='toggle_btn', width='auto'),
        # layout=widgets.Layout(width='150px')
    )
    toggle_btn.sensor_id = sensor_id
    toggle_btn.style.font_weight = 'bold'

    # Create a Label to display the timer next to the button
    timer_label = widgets.Label(
        value="Time: 00:00:00", 
        layout=widgets.Layout(
            grid_area='timer_label', width='auto',
        )
    )
    timers.append(timer_label)

    # Define the observer for the toggle button.
    def on_toggle(change, btn=toggle_btn, lbl=timer_label):
        if change['new']:
            btn.description = f"Sensor {btn.sensor_id}: Stop"
            sensor_recording(btn.sensor_id, True, lbl)
        else:
            btn.description = f"Sensor {btn.sensor_id}: Start"
            sensor_recording(btn.sensor_id, False, lbl)

    toggle_btn.observe(on_toggle, names='value')

    # Create the sensor-specific test button.
    test_btn = widgets.Button(
        description="Test",
        button_style='info',
        layout=widgets.Layout(grid_area='test_btn', width='auto'),
        # layout=widgets.Layout(width='100px')
    )
    test_btn.sensor_id = sensor_id
    test_btn.on_click(sensor_test)

    start_vol_tinput = widgets.BoundedFloatText(
        value=0,
        min=0,
        max=100.0,
        step=0.01,
        layout=widgets.Layout(grid_area="start_vol_tinput", width='auto'),
    )
    start_vol_tinput.sensor_id = sensor_id
    stop_vol_tinput = widgets.BoundedFloatText(
        value=0,
        min=0,
        max=100.0,
        step=0.01,
        layout=widgets.Layout(grid_area="stop_vol_tinput", width='auto'),
    )
    stop_vol_tinput.sensor_id = sensor_id

    # Labels for the volume recording widgets
    start_column_footer = widgets.Label(value="Start Vol", layout=widgets.Layout(grid_area="start_column_footer", width='auto', height='auto'))
    stop_column_footer = widgets.Label(value="Stop Vol", layout=widgets.Layout(grid_area="stop_column_footer", width='auto', height='auto'))
    
    # Combine the toggle and test buttons with the volume input boxes into a horizontal box.
    grid_layout = widgets.Layout(
        width="16%",
        grid_template_rows="40% 20% 40%",
        justify_content='flex-start',
        border='1px solid black',
        grid_template_areas='''
        "toggle_btn toggle_btn toggle_btn test_btn"
        "start_vol_tinput stop_vol_tinput timer_label timer_label"
        "start_column_footer stop_column_footer timer_label timer_label"
        '''
    )
    button_group = widgets.GridBox(children=[toggle_btn, test_btn, start_column_footer, stop_column_footer, timer_label, start_vol_tinput, stop_vol_tinput], layout=grid_layout)
    button_groups.append(button_group)


# --- Arrange the buttons into rows ---
# I'm hardcoding the UI to display 24 sensors for now. This is the maximum number of cages we will currently be able
# to accomodate on the rack and we aren't doing dual-channels with the MPR121s. This will keep things consistent
# between runs, hopefully

rows = []
# Loop over the 4 shelves of the rack,
for shelf in range(4):
    # Determine which sensors to add to the row
    sensors_subset = np.array([1, 2, 3, 4, 5, 6]) + 6*shelf
    row = []
    for sensor in sensors_subset:
        # The button_groups list is 0-indexed so -1 on the index
        button_group = button_groups[sensor-1]
        row.append(button_group)
    row = widgets.HBox(row)
    rows.append(row)
sensor_rows = widgets.VBox(rows)

# --- Create the Clear Output/Graph Buttons ---
clear_output_button = widgets.Button(
    description="Clear Output",
    button_style='warning'
)
# clear_output_button.layout.width = '150px'
# clear_output_button.layout.height = '40px'
clear_output_button.on_click(clear_output_callback)

clear_graph_button = widgets.Button(
    description="Clear Graph",
    button_style='warning'
)
# Set the layout properties after creation to avoid JSON serialization issues.
# clear_graph_button.layout.width = '150px'
# clear_graph_button.layout.height = '40px'
clear_graph_button.on_click(clear_graph_callback)

clear_buttons = widgets.HBox([
    clear_output_button, clear_graph_button,
])

# --- Combine Everything into the Final Layout ---
final_ui = widgets.VBox([
    all_devices_button,  # Top large start/stop-all button
    sensor_rows,     # Sensor grid arranged in 3 columns
    clear_buttons,       # Buttons to clear the output/graph widgets
    outputs_hbox,        # Output/graph widgets side-by-side
])

# Display the complete UI.
display(final_ui)
